# Current_IPOs_GMP_Code

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fetch the webpage
url = "https://www.investorgain.com/report/live-ipo-gmp/331/current/"
response = requests.get(url)
# Parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")

# Find all tables on the page
tables = soup.find_all("table")
# Extract data from the first table
table_data = []
if len(tables) >= 1:
    first_table = tables[0]
    rows = first_table.find_all("tr")
    for row in rows:
        cells = row.find_all(["td", "th"])
        row_data = [cell.text.strip() for cell in cells]
        if row_data:
            table_data.append(row_data)
# Convert the data to a pandas DataFrame
df = pd.DataFrame(table_data[1:], columns=table_data[0])
# Define the file path for exporting to desktop
#desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
#file_path = os.path.join(desktop_path, "Current_IPOs.csv")
# Export DataFrame to CSV
#df.to_csv(file_path, index=False)
#print(f"Data exported to {file_path}")
df1 = pd.read_csv("desktop/IPO/Current_IPOs.csv")
df.to_csv('desktop/IPO/Current_IPOs.csv', index=False)
##Data Cleaning
df1.drop(columns=["Fire Rating","IPO Size", "Est Listing"], inplace=True)
df1.dropna(subset=["Price"], inplace=True)
df1.replace("--", 0, inplace=True)
df1["Lot"] = pd.to_numeric(df1["Lot"], errors='coerce')
df1.rename(columns={"GMP(₹)": "GMP"}, inplace=True)
df1 = df1[df1["IPO"].str.contains('Open|Today', case=False)]
df1['IPO_Name'] = df1['IPO'].str.split().str[:2].str.join(' ')
df1['IPOfirstname'] = df1['IPO'].str.split().str[0]
df1["Amount"] = df1["Price"]* df1["Lot"]
df1["Profit"] = df1["GMP"].astype(float) * df1["Lot"]

# Create a new field based on "IPO" column
df1['IPO_Type'] = df1['IPO'].apply(lambda x: 'SME' if 'SME' in x else 'Mainboard')
df1.to_csv(r'C:\Users\vjha8\Desktop\IPO\cleaned_df.csv', index=False)

# Mainboad IPOs Subscription Status

In [2]:
# URL of the webpage
url = "https://www.chittorgarh.com/report/ipo-subscription-status-live-bidding-data-bse-nse/21/"
# Send a GET request to the URL
response = requests.get(url)
# Parse HTML content
soup = BeautifulSoup(response.content, "html.parser")
# Find the second table on the webpage
table = soup.find_all("table")[2]  # Index 1 for the second table
# Extract data from the table
data = []
for row in table.find_all("tr"):
    row_data = [cell.get_text(strip=True) for cell in row.find_all(["th", "td"])]
    data.append(row_data)
# Convert data to DataFrame
df = pd.DataFrame(data[1:], columns=data[0])
if "Applications" in df.columns:
    df.drop(columns=["Applications"], inplace=True)
if "Size (Rs Cr)" in df.columns:
    df.drop(columns=["Size (Rs Cr)"], inplace=True)
if "Close Date" in df.columns:
    df.drop(columns=["Close Date"], inplace=True)
# Export DataFrame to CSV
df.to_csv(r'C:\Users\vjha8\Desktop\IPO\Mainboard_IPOs_subs_Status.csv', index=False)
print("Mainboard_IPOs_subs_Status downloaded at desktop")

Mainboard_IPOs_subs_Status downloaded at desktop


# SME IPOs Subscription Status

In [3]:
# URL of the webpage
url = "https://www.chittorgarh.com/report/ipo-subscription-status-live-bidding-data-bse-nse/22/"
# Send a GET request to the URL
response = requests.get(url)
# Parse HTML content
soup = BeautifulSoup(response.content, "html.parser")
# Find the second table on the webpage
table = soup.find_all("table")[2]  # Index 1 for the second table
# Extract data from the table
data = []
for row in table.find_all("tr"):
    row_data = [cell.get_text(strip=True) for cell in row.find_all(["th", "td"])]
    data.append(row_data)
# Convert data to DataFrame
df = pd.DataFrame(data[1:], columns=data[0])
if "Applications" in df.columns:
    df.drop(columns=["Applications"], inplace=True)
if "Size (Rs Cr)" in df.columns:
    df.drop(columns=["Size (Rs Cr)"], inplace=True)
if "Open Date" in df.columns:
    df.drop(columns=["Open Date"], inplace=True)
if "Close Date" in df.columns:
    df.drop(columns=["Close Date"], inplace=True)
# Export DataFrame to CSV
df.to_csv(r'C:\Users\vjha8\Desktop\IPO\SME_IPOs_subs_Status.csv', index=False)
print("SME_IPOs_subs_Status downloaded at desktop")

SME_IPOs_subs_Status downloaded at desktop


# Final data 

In [4]:
import pandas as pd

# Read the CSV files
sme = pd.read_csv('Desktop\IPO\SME_IPOs_subs_Status.csv')
main = pd.read_csv('Desktop\IPO\Mainboard_IPOs_subs_Status.csv')
cleaned = pd.read_csv('Desktop\IPO\cleaned_df.csv')

# Fill null values with 0
sme.fillna(0, inplace=True)
main.fillna(0, inplace=True)

# Clean 'Company Name' column
sme['Company Name'] = sme['Company Name'].str.replace('-', ' ')
main['Company Name'] = main['Company Name'].str.replace('-', ' ')

# Extract 'firstname' from 'Company Name'
sme['firstname'] = sme['Company Name'].str.split(n=1).str[0]
main['firstname'] = main['Company Name'].str.split(n=1).str[0]

# Select only the required columns from 'sme' and 'main'
sme_subset = sme[['firstname', 'QIB (x)', 'NII (x)', 'Retail (x)']]
main_subset = main[['firstname', 'QIB (x)', 'NII (x)', 'Retail (x)']]

# Merge 'cleaned' with 'sme_subset' first
merged_df = pd.merge(cleaned, sme_subset, how='left', left_on='IPOfirstname', right_on='firstname')

# Merge 'cleaned' with 'main_subset' separately
main_merged = pd.merge(cleaned, main_subset, how='left', left_on='IPOfirstname', right_on='firstname')

# Combine the results, preferring sme values if available
for col in ['QIB (x)', 'NII (x)', 'Retail (x)']:
    merged_df[col] = merged_df[col].combine_first(main_merged[col])

# Drop the duplicate 'firstname' column from the first merge
merged_df.drop(columns=['firstname'], inplace=True)

# Save the merged DataFrame to Excel
merged_df.to_excel(r'C:\Users\vjha8\Desktop\IPO\finaldata.xlsx', index=False)
print("Finaldata downloaded at desktop\IPO")

Finaldata downloaded at desktop\IPO


# Importing data on Google Sheet

In [5]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Load data from CSV files
sme = pd.read_csv('Desktop/IPO/SME_IPOs_subs_Status.csv')
main = pd.read_csv('Desktop/IPO/Mainboard_IPOs_subs_Status.csv')
cleaned = pd.read_csv('Desktop/IPO/cleaned_df.csv')

# Preprocess data
sme['Company Name'] = sme['Company Name'].str.replace('-', ' ')
sme['firstname'] = sme['Company Name'].str.split(n=1).str[0]
main['Company Name'] = main['Company Name'].str.replace('-', ' ')
main['firstname'] = main['Company Name'].str.split(n=1).str[0]

sme.fillna(0, inplace=True)
main.fillna(0, inplace=True)

# Selecting only the required columns from 'sme' and 'main'
sme_subset = sme[['QIB (x)','NII (x)','Retail (x)','Total (x)','firstname']]
main_subset = main[['QIB (x)','NII (x)','Retail (x)','Total (x)','firstname']]


merged_df = pd.merge(cleaned, sme_subset, how='left', left_on='IPOfirstname', right_on='firstname')
merged_df.fillna(0, inplace=True)
# Merging 'cleaned' with 'main_subset' separately
main_merged = pd.merge(cleaned, main_subset, how='left', left_on='IPOfirstname', right_on='firstname')


# Combine the results, preferring sme values if available
for col in ['QIB (x)', 'NII (x)', 'Retail (x)']:
    if col in merged_df.columns and col in main_merged.columns:
        merged_df[col] = merged_df[col].combine_first(main_merged[col])
merged_df.drop(columns=['firstname'], inplace=True)

# Authenticate with Google Sheets API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'C:\Users\vjha8\Downloads\authfile.json', scope)
client = gspread.authorize(credentials)

# Open the specific spreadsheet by its title
spreadsheet = client.open('IPO')

# Select the worksheet where you want to update the data
worksheet = spreadsheet.get_worksheet(0)  # Replace 0 with the index of your worksheet

# Clear existing data from the worksheet
worksheet.clear()

# Convert DataFrame to a list of lists (2D array) for easy upload
data = [merged_df.columns.tolist()] + merged_df.values.tolist()

# Append data to the worksheet
worksheet.append_rows(data)

print("Data has been successfully pushed to the Google Sheet.")


Data has been successfully pushed to the Google Sheet.
